In [ ]:
#idk diference between cv and cv2. I think cv2 has class functionality
#import cv

In [26]:
#library needed on beaglebone
import cv2
import numpy as np
import time

In [27]:
#import matplotlib
import matplotlib.pyplot as plt

In [88]:
def get_480p(file):
    img=cv2.imread(file)
    img=cv2.resize(img,(640,480))
    gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img,gray_img
def get_cut_480p(file,xcut=[0,640],ycut=[0,480]):
    img=cv2.imread(file)
    img=cv2.resize(img,(640,480))
    img1=img[ycut[0]:ycut[1],xcut[0]:xcut[1]]
    gray_img=cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    return img,img1,gray_img
def get_360p(file):
    img=cv2.imread(file)
    img=cv2.resize(img,(320,240))
    gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img,gray_img
def display(img):
    try:
        b,g,r = cv2.split(img)
        img2 = cv2.merge([b,g,r])
        imgplot = plt.imshow(img)
        plt.show()
    except:    
        imgplot = plt.imshow(img,cmap='gray')
        plt.show()
def hough_trans(img,gray,can_min=50,can_max=350,vote=100,plot_canny=0,plot_img=0):
    t1 = time.time()
    edges = cv2.Canny(gray,can_min,can_max,apertureSize = 3)
    t2 = time.time()
    lines = cv2.HoughLines(edges,1,np.pi/180,vote)
    t3 = time.time()
    print 'canny',t2 - t1
    print 'hough',t3 - t2
    if plot_canny==1:
                display(edges)
    #print lines
    if plot_img==1:
        try:
            for rho,theta in lines[0]:
                print rho,theta
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
                cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
            display(img)
        except:
            print('could not plot')
    else:
        try:
            return len(lines[0])
        except:
            return 0
def hough_transp(img,gray,can_min=150,can_max=300,vote=150,
                 minLineLength = 20,maxLineGap = 15,plot_canny=0,plot_img=0):
    #hough that uses less test points, less computations
    t1 = time.time()
    edges = cv2.Canny(gray,can_min,can_max,apertureSize = 3)
    t2 = time.time()
    lines = cv2.HoughLinesP(edges,1,np.pi/180,vote,minLineLength,maxLineGap)
    t3 = time.time()
    print 'canny',t2 - t1
    print 'hough',t3 - t2
    if plot_canny==1:
        display(edges)
    #print lines
    if plot_img==1:
        try:
            for x1,y1,x2,y2 in lines[0]:
                cv2.line(img,(x1,y1),(x2,y2),(255,0,0),2)
            display(img)
        except:
            print('could not plot')
    else:
        try:
            return len(lines[0])
        except:
            return 0
def hough_trans_blur(img,gray,can_min=50,can_max=350,vote=100,plot_canny=0):
    t1 = time.time()
    blur = cv2.blur(gray,(2,2))
    t2 = time.time()
    edges = cv2.Canny(gray,can_min,can_max,apertureSize = 3)
    t3 = time.time()
    lines = cv2.HoughLines(edges,1,np.pi/180,vote)
    t4 = time.time()
    print 'blur', t2 - t1
    print 'canny',t3 - t2
    print 'hough',t4 - t3
    #print lines
    for rho,theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
    if plot_canny==1:
        display(edges)
    display(img)

In [42]:
#look at white concrete
img,reimg,gray=get_cut_480p('9.jpg',xcut=[400,470],ycut=[420,480])#get img, roi image, and gray cut img
#img,reimg,gray=get_cut_480p('1.jpg',xcut=[600,640],ycut=[400,480])#get img, roi image, and gray cut img
display(reimg)
b,g,r = cv2.split(reimg)
print reimg
#hsv to detect white?

In [126]:
#create green pointers on sidewalk
img,reimg,gray=get_cut_480p('9.jpg',xcut=[0,640],ycut=[0,480])#get img, roi image, and gray cut img
#display(reimg)
#b,g,r = cv2.split(reimg)
i=0;
for x in range(0,640,10):#
    for y in range(0,480,10):
        bgr=reimg[y,x]
        #also take the difference between g and r? if too high means its not white
        if bgr[0]*1.2>(bgr[1]+bgr[2])/2:#check if white with a bit more of blue
        #if abs(bgr[1]-bgr[2])<15:
            i+=1
            #draw where true
            for x1 in range(-2,3):
                for y1 in range(-2,3):
                    reimg[y+y1,x+x1]=[0,200,255]
print i
#hsv to detect white?
display(reimg)
cv2.imwrite('9_white_detection.jpg', reimg)

2626


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: RuntimeWarning: overflow encountered in ubyte_scalars


True

In [105]:
print reimg[380:420,280:300]
display(reimg[380:420,280:300])

[[[ 55  71  77]
  [ 56  72  78]
  [ 57  73  79]
  ..., 
  [ 72  91  94]
  [255 200   0]
  [255 200   0]]

 [[ 52  68  74]
  [ 53  69  75]
  [ 52  68  74]
  ..., 
  [ 78  97 100]
  [255 200   0]
  [255 200   0]]

 [[ 52  69  73]
  [ 53  69  75]
  [ 55  71  77]
  ..., 
  [ 47  66  69]
  [255 200   0]
  [255 200   0]]

 ..., 
 [[ 61  80  84]
  [ 67  86  89]
  [ 76  95  98]
  ..., 
  [ 66  85  88]
  [ 69  88  91]
  [ 77  96  99]]

 [[ 62  79  84]
  [ 67  86  89]
  [ 70  89  92]
  ..., 
  [ 57  76  79]
  [ 66  85  88]
  [ 82 101 104]]

 [[ 60  76  82]
  [ 67  86  89]
  [ 70  89  92]
  ..., 
  [ 73  92  96]
  [ 74  93  96]
  [ 78  97 100]]]


In [42]:
#find corners
img,gray=get_480p('1.jpg')

#gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]

display(img)

In [13]:
img1,gray=get_480p('2-knee.jpg')
display(img1)
#blur = cv2.bilateralFilter(gray,9,75,75)
#display(blur)
hough_trans(img1,gray,vote=220,can_min=150,can_max=250)

canny 0.0295388698578
hough 0.0421450138092


1

In [99]:
#iterating hough
img,reimg,gray=get_cut_480p('1.jpg',xcut=[240,520],ycut=[200,480])#get img, roi image, and gray cut img
display(reimg)
#img1,gray=get_360p('2-knee.jpg')
it_vote=100
lvote=[it_vote]
lln=[]
mu=.75
n_lines=5
for x in range(0,5):
    ln=hough_trans(reimg,gray,vote=int(it_vote),can_min=200,can_max=250)
    lln.append(ln)
    fixed=n_lines-ln
    #iterate
    it_vote=it_vote-mu*fixed
    lvote.append(it_vote)
ln=hough_trans(reimg,gray,vote=int(it_vote),can_min=200,can_max=250,plot_canny=1,plot_img=1)
print(lvote)
plt.figure(1)
plt.plot(lvote)
plt.ylabel('Vote')
plt.xlabel('Iteration')
plt.figure(2)
plt.plot(lln)
plt.ylabel('Number of lines')
plt.xlabel('Iteration')
plt.show()

canny 0.00276684761047
hough 0.0175821781158
canny 0.0019679069519
hough 0.0138740539551
canny 0.00164484977722
hough 0.0108580589294
canny 0.00142383575439
hough 0.0124080181122
canny 0.00128293037415
hough 0.00853610038757
canny 0.00126194953918
hough 0.00887513160706
51.0 1.62316
53.0 1.62316
-26.0 3.00197
50.0 1.64061
188.0 1.6057
3.0 1.62316
191.0 1.6057
5.0 1.62316
-23.0 2.98451
-24.0 3.00197
[100, 107.5, 113.5, 118.75, 122.5, 126.25]


In [61]:
img,gray=get_cut_480p('4-knee.jpg')
display(img)
it_vote=200
lvote=[it_vote]
lln=[]
mu=.45
n_lines=2
for x in range(0,20):
    ln=hough_trans(img1,gray,vote=int(it_vote),can_min=200,can_max=250)
    lln.append(ln)
    fixed=n_lines-ln
    #iterate
    it_vote=it_vote-mu*fixed
    lvote.append(it_vote)
ln=hough_trans(img1,gray,vote=int(it_vote),can_min=200,can_max=250,plot_canny=1,plot_img=1)
print(lvote)
plt.figure(1)
plt.plot(lvote)
plt.ylabel('Vote')
plt.xlabel('Iteration')
plt.figure(2)
plt.plot(lln)
plt.ylabel('Number of lines')
plt.xlabel('Iteration')
plt.show()

ValueError: too many values to unpack

In [21]:
#circles
img,gray=get_480p('1-a.jpg')
display(gray)
blur = cv2.medianBlur(gray,5)
display(blur)
circles = cv2.HoughCircles(blur,3,30,3)

circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(gray,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(gray,(i[0],i[1]),2,(0,0,255),3)
display(gray)

In [ ]:
#get low image resolution or take midsection of high res.
#blur, use the bit operator to only output where the sidewalk is, travel on right side.
#in forest area use the high intensity light as a sign of the sidewalk being there
#perhaps make algorithim to detect white

In [ ]:
#try to use the averaging filters to get the whole concrete slab
img,gray=get_480p('8.jpg')
blur = cv2.blur(gray,(2,2))
display(blur)
hough_trans(img,gray)

In [ ]:
#use bilateral filtering to remove texture(concrete) but is cpu intensive. 
img,gray=get_480p('8.jpg')
blur = cv2.bilateralFilter(gray,9,75,75)
display(blur)
hough_trans(img,gray)

In [ ]:
#individual colors
img,gray=get_480p('4-knee.jpg')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower_blue = np.array([50,50,110])
upper_blue = np.array([255,255,130])
mask = cv2.inRange(hsv, lower_blue, upper_blue)
 # Bitwise-AND mask and original image
res = cv2.bitwise_and(img,img, mask= mask)
display(img)
display(mask)
display(res)

In [ ]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img=cv2.imread('4-knee.jpg')

In [ ]:
#returns (height, width, pixel element number)
height, width, pixel=img.shape
print img.shape

In [ ]:
imgplot = plt.imshow(img)
plt.show()

In [ ]:
img2 = img[280:340, 330:390]#take section of the picture
img[273:333, 100:160] = img2#put that section on original picutre

In [ ]:
imgplot = plt.imshow(img)
plt.show()

In [ ]:
b,g,r = cv2.split(img)#full splitting into colors
b = img[:,:,0]#individual blue color
imgplot = plt.imshow(b)
plt.show()

In [ ]:
#Grayscale
gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgplot = plt.imshow(gray_img)
plt.show()

In [ ]:
#HSV
hsv_img=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
imgplot = plt.imshow(hsv_img)
plt.show()

In [ ]:
print img[0,0]
print gray_img[0,0]
print hsv_img[0,0]

In [ ]:
#capture video
cap = cv2.VideoCapture(0)
while 1:
    ret, frame = cap.read()
    if ret==True:#check if cap.read() returned frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        display(gray)
        #image recognition code here
    break

In [ ]:
#canny edge detection
img = cv2.imread('4-knee.jpg',0)
edges = cv2.Canny(img,100,200)

plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
img = cv2.imread('1-a.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,160,300,apertureSize = 3)
lines = cv2.HoughLines(edges,1,np.pi/180,200)
#print lines
for rho,theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

plt.subplot(121),plt.imshow(edges,cmap = 'gray')
plt.subplot(122),plt.imshow(img,cmap = 'gray')
#plt.imshow(img,cmap='gray')
plt.show()
#cv2.imwrite('houghlines1.jpg',img)

In [ ]:
#hough that uses less test points, less computations
img = cv2.imread('4-knee.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)
minLineLength = 100
maxLineGap = 20
lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)
#print lines
for x1,y1,x2,y2 in lines[0]:
    cv2.line(img,(x1,y1),(x2,y2),(255,0,0),2)
plt.subplot(121),plt.imshow(edges,cmap = 'gray')
plt.subplot(122),plt.imshow(img,cmap = 'gray')
plt.show()
#cv2.imwrite('houghlines2.jpg',img)